In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-eu.apache.org/dist/spark/spark-2.3.3/spark-2.3.3-bin-hadoop2.7.tgz
!tar xf spark-2.3.3-bin-hadoop2.7.tgz
!pip install -q findspark

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.3-bin-hadoop2.7"

In [0]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

In [0]:
spark = SparkSession.builder.appName('cluster').getOrCreate()

In [0]:
from pyspark.ml.clustering import KMeans

In [0]:
dataset = spark.read.format('libsvm').load('sample_kmeans_data.txt')

In [8]:
dataset.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|           (3,[],[])|
|  1.0|(3,[0,1,2],[0.1,0...|
|  2.0|(3,[0,1,2],[0.2,0...|
|  3.0|(3,[0,1,2],[9.0,9...|
|  4.0|(3,[0,1,2],[9.1,9...|
|  5.0|(3,[0,1,2],[9.2,9...|
+-----+--------------------+



In [0]:
final_data = dataset.select('features')

In [0]:
kmeans = KMeans().setK(3).setSeed(1)

In [0]:
model = kmeans.fit(final_data)

In [0]:
# within set sum of squared errors
wssse = model.computeCost(final_data)

In [25]:
print(wssse)

0.07499999999994544


In [0]:
# getting the centroids
centers = model.clusterCenters() 

In [27]:
centers

[array([9.1, 9.1, 9.1]), array([0.05, 0.05, 0.05]), array([0.2, 0.2, 0.2])]

In [0]:
results = model.transform(final_data)

In [29]:
results.show()

+--------------------+----------+
|            features|prediction|
+--------------------+----------+
|           (3,[],[])|         1|
|(3,[0,1,2],[0.1,0...|         1|
|(3,[0,1,2],[0.2,0...|         2|
|(3,[0,1,2],[9.0,9...|         0|
|(3,[0,1,2],[9.1,9...|         0|
|(3,[0,1,2],[9.2,9...|         0|
+--------------------+----------+



In [0]:
spark = SparkSession.builder.appName('cluster').getOrCreate()

In [0]:
dataset = spark.read.csv('seeds_dataset.csv',inferSchema=True,header=True)

In [32]:
dataset.printSchema()

root
 |-- area: double (nullable = true)
 |-- perimeter: double (nullable = true)
 |-- compactness: double (nullable = true)
 |-- length_of_kernel: double (nullable = true)
 |-- width_of_kernel: double (nullable = true)
 |-- asymmetry_coefficient: double (nullable = true)
 |-- length_of_groove: double (nullable = true)



In [33]:
dataset.head(1)

[Row(area=15.26, perimeter=14.84, compactness=0.871, length_of_kernel=5.763, width_of_kernel=3.312, asymmetry_coefficient=2.221, length_of_groove=5.22)]

In [0]:
from pyspark.ml.clustering import KMeans

In [0]:
from pyspark.ml.feature import VectorAssembler

In [36]:
dataset.columns

['area',
 'perimeter',
 'compactness',
 'length_of_kernel',
 'width_of_kernel',
 'asymmetry_coefficient',
 'length_of_groove']

In [0]:
assembler = VectorAssembler(inputCols=dataset.columns,outputCol='features')

In [0]:
final_data = assembler.transform(dataset)

In [39]:
final_data.printSchema()

root
 |-- area: double (nullable = true)
 |-- perimeter: double (nullable = true)
 |-- compactness: double (nullable = true)
 |-- length_of_kernel: double (nullable = true)
 |-- width_of_kernel: double (nullable = true)
 |-- asymmetry_coefficient: double (nullable = true)
 |-- length_of_groove: double (nullable = true)
 |-- features: vector (nullable = true)



In [0]:
# feature scaling
from pyspark.ml.feature import StandardScaler

In [0]:
scaler = StandardScaler(inputCol='features',outputCol='scaledFeatures')

In [0]:
scaler_model = scaler.fit(final_data)

In [0]:
final_data = scaler_model.transform(final_data)

In [45]:
final_data.head(1)

[Row(area=15.26, perimeter=14.84, compactness=0.871, length_of_kernel=5.763, width_of_kernel=3.312, asymmetry_coefficient=2.221, length_of_groove=5.22, features=DenseVector([15.26, 14.84, 0.871, 5.763, 3.312, 2.221, 5.22]), scaledFeatures=DenseVector([5.2445, 11.3633, 36.8608, 13.0072, 8.7685, 1.4772, 10.621]))]

In [0]:
kmeans = KMeans(featuresCol='scaledFeatures',k=3)

In [0]:
model = kmeans.fit(final_data)

In [0]:
wssse = model.computeCost(final_data)

In [49]:
print(wssse)

428.85343407938126


In [50]:
centers = model.clusterCenters()
print(centers)

[array([ 4.91309043, 10.92012526, 37.32658724, 12.37724251,  8.59393872,
        1.8226071 , 10.35389957]), array([ 6.32636687, 12.38115343, 37.39222755, 13.9206997 ,  9.75485787,
        2.41428142, 12.28078861]), array([ 4.0648023 , 10.14242485, 35.82143905, 11.81918014,  7.51855717,
        3.19361875, 10.40520609])]


In [51]:
model.transform(final_data).select('prediction').show()

+----------+
|prediction|
+----------+
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         1|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         2|
+----------+
only showing top 20 rows

